## ランダムフォレスト　デフォルト

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import tree


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
train_pkl.shape

(55323, 12)

In [4]:
train_pkl.describe()

,id,accommodates,bathrooms,bedrooms,beds,host_response_rate,number_of_reviews,y,bed_type_flag,property_type_num,cancel_policy_flag,city_flag
count,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,5.532300e+04,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000,55323.000000
mean,27787.181588,0.087360,0.248414,0.111274,0.238273,2.311836e-17,0.204165,160.307341,1.950943,0.999259,0.001826,0.295103
std,16043.404061,0.790027,0.782466,0.794160,0.712608,1.000009e+00,0.752675,168.266655,0.295932,0.075455,0.047878,0.616145
min,0.000000,-1.000000,-2.000000,-1.000000,-1.000000,-1.651052e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,13892.500000,0.000000,0.000000,0.000000,0.000000,-1.651052e+00,0.000000,74.000000,2.000000,1.000000,0.000000,0.000000
50%,27778.000000,0.000000,0.000000,0.000000,0.000000,6.706177e-01,0.000000,111.000000,2.000000,1.000000,0.000000,0.000000
75%,41681.500000,0.000000,0.000000,0.000000,0.000000,6.706177e-01,0.000000,185.000000,2.000000,1.000000,0.000000,0.000000
max,55582.000000,5.000000,11.000000,10.000000,12.000000,6.706177e-01,15.000000,1999.000000,2.000000,2.000000,2.000000,2.000000


## 訓練データとテストデータに分割

In [5]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [6]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

## 訓練データで訓練

In [7]:
RF = RandomForestRegressor(random_state=42)

In [8]:
RF = RF.fit(train_X, train_y)

In [9]:
RF.feature_importances_

array([0.13753375, 0.19790574, 0.39055573, 0.06173143, 0.1091081 ,
       0.01251814, 0.00674091, 0.00807474, 0.00207754, 0.07375393])

In [10]:
sorted(
    zip(map(lambda x: round(x, 3), RF.feature_importances_), train.iloc[:, [1, 2, 3, 4, 5, 6, 8, 9, 10, 11]].columns),
    reverse=True)

[(0.391, 'bedrooms'),
 (0.198, 'bathrooms'),
 (0.138, 'accommodates'),
 (0.109, 'host_response_rate'),
 (0.074, 'city_flag'),
 (0.062, 'beds'),
 (0.013, 'number_of_reviews'),
 (0.008, 'property_type_num'),
 (0.007, 'bed_type_flag'),
 (0.002, 'cancel_policy_flag')]

In [11]:
# 訓練済みの決定木を視覚化
# dot_data = tree.export_graphviz(RF, out_file=None,
#                                feature_names=train.iloc[:, 1:].columns,
#                                class_names=train.Survived.name,
#                                rounded=True,
#                                filled=True,
#                                special_characters=True)

## テストデータで実行

In [12]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [13]:
test_x.shape, test_y.shape

((11065, 10), (11065,))

In [14]:
pred_y = RF.predict(test_x)

In [15]:
np.sqrt(mean_squared_error(test_y, pred_y))

134.6330955005463

## 検証データで実行

In [ ]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [ ]:
valid.shape

In [ ]:
# ID の保存
valid_pass = valid.id.values

In [ ]:
valid_X = valid.iloc[:, 1:].values

In [ ]:
# valid_X.describe()

In [ ]:
valid_X.shape, train_X.shape

In [ ]:
pred_valid_y = RF.predict(valid_X)

In [ ]:
pred_valid_y.shape

In [ ]:
type(valid_pass), type(pred_valid_y)

In [ ]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [ ]:
result_df.head()

In [ ]:
result_df.to_csv("./RF_8.csv", header=False)